Setup

In [ ]:
%%bash
apt install bedtools
which intersectBed
git clone https://github.com/ysut/2024_CellGenmics.git
pip install gffutils pandas pandarallel pysam tqdm biopython pybedtools liftover
anno_db="https://zenodo.org/records/13310274/files/gencode.v43lift37.annotation.gtf.db"
anno_int_db="https://zenodo.org/records/13310274/files/gencode.v43lift37.annotation.intron.gtf.db"
wget -c -P /content/2024_CellGenomics/resources/06_gffutilsdb "${anno_db}"
wget -c -P /content/2024_CellGenomics/resources/06_gffutilsdb "${anno_int_db}"

Analysis

In [ ]:
import os
os.chdir('/content/2024_CellGenomics/')
import yaml
from logging import getLogger, config

import gffutils
import pandas as pd
from pandarallel import pandarallel
import pysam
from tqdm import tqdm

from lib import utils, posparser, splaiparser, predeffect, anno_clinvar
from lib.scoring import Scoring

########   Initialize and setup pandas methods   ########
pandarallel.initialize(
    nb_workers=os.cpu_count()-1, progress_bar=False, verbose=2, use_memory_fs=False) 
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp' 
tqdm.pandas()

########   Logging setup   ########
parent_directory = os.path.dirname(os.path.dirname('__file__'))
config_path: str = os.path.join(parent_directory, 'config/logging.yaml')
with open(config_path, 'r') as f:
    config.dictConfig(yaml.safe_load(f))
logger = getLogger(__name__)

In [30]:
########   Import genocode DBs (exon DB and intron DB)   ########
db_anno_gencode = 'resources/06_gffutilsdb/gencode.v43lift37.annotation.gtf.db'
db_anno_intron = 'resources/06_gffutilsdb/gencode.v43lift37.annotation.intron.gtf.db'
db = gffutils.FeatureDB(db_anno_gencode)
db_intron = gffutils.FeatureDB(db_anno_intron)

########   Import resource files   ########
#1. Clinvar variants (BED format)
clinvar_file = 'resources/03_ClinVar/variant_summary.snv.grch37.germline.criteria.sort.bed.gz'
tbx_clinvar = pysam.TabixFile(clinvar_file)
#2. GENCODE file (GFF3 format)
gencode_gff = 'resources/05_GENCODE_v43lift37/gencode.v43lift37.annotation.sort.gff3.gz'
tbx_anno = pysam.TabixFile(gencode_gff)

## Thresholds configuration
thresholds_SpliceAI_parser: dict = {
    'TH_min_sALDL': 0.02, 'TH_max_sALDL': 0.2, 
    'TH_min_sAGDG': 0.01, 'TH_max_sAGDG': 0.05,
    'TH_min_GExon': 25, 'TH_max_GExon': 500,
    'TH_sAG': 0.2, 'TH_sDG': 0.2
    }

sccore_ths = {'clinvar_same_pos': 2,     
              'clinvar_same_motif': 1,
              'clinvar_else': 0,
              'non_canon_splai_lte_0.1_outside': -3,
              'non_canon_splai_lte_0.1_other': -2,
              'non_canon_splai_bet_0.1_0.2': 1,
              'non_canon_splai_gte_0.2': 2,
              'canon_strong': 6, 
              'canon_moderate': 5, 
              'frameshift_nmd_eloF': 7, 
              'frameshift_nmd_not_eloF': 3,
              'canon_splai_lte_0.1': -3,
              'canon_splai_bet_0.1_0.2': -1,
              'canon_splai_gte_0.2': 0}

# Two example pickle files are provided in the 'cohort' folder
mydata_pickle = 'cohort/example_cases_1.pkl'
#mydata_pickle = 'cohort/example_cases_2.pkl'

De novo detection

In [31]:
df = pd.read_pickle(mydata_pickle)
df = df[df['vqslod'] > -7.18]
df = df[((df['denovogear'] > 0.02) | (df['denovogear'].isnull()))
        & ((df['triodenovo'] > 5.72) | (df['triodenovo'].isnull()))
        & ((df['dnmfilter'] > 0.196) | (df['dnmfilter'].isnull()))]

Classify 'Canonical' splice site or 'Non-canonical' splice site

In [ ]:
logger.info('Classify "Canonical" splice site or "Non-canonical" splice site...')
df = posparser.classifying_canonical(df, cdot='c.HGVS')

Calculate exonic positions

In [ ]:
logger.info('Calculating exonic positions...')

#2-1. Generate 'exonic upstream distance and exonic downstream distance
df['exon_loc'] = df.progress_apply(
    posparser.calc_exon_loc, tabixfile=tbx_anno, enstcolname='ENST', axis=1)
df = pd.concat([df, df['exon_loc'].str.split(':', expand=True)], axis=1)
df.rename(columns={0: 'ex_up_dist', 1: 'ex_down_dist'}, inplace=True)

#2-2. Select minimum distance from upstream distance and downstream distance
df['exon_pos'] = df.parallel_apply(posparser.select_exon_pos, axis=1)

#2-3. Decision exonic splice sites (1 nt in acceptor site or 3 nts on Donor site)
df['exon_splice_site'] = df.parallel_apply(
    posparser.extract_splicing_region, axis=1)

Additional Splicing information

In [ ]:
logger.info('Annotating splicing information...')

#3-1. Annotate splicing type ('Exonic Acceptor' etc.)
df['SpliceType'] = df.parallel_apply(
    posparser.select_donor_acceptor, axis=1)

#3-2. Annotate rank of exon or intron
df['Num_ExInt'] = df.progress_apply(
    posparser.calc_ex_int_num, db=db, db_intron=db_intron, axis=1)

Annotate ClinVar varaints interpretations

In [ ]:
logger.info('Annotating ClinVar varaints interpretations...')
df['clinvar_same_pos'] = df.progress_apply(
    anno_clinvar.anno_same_pos_vars, tabixfile=tbx_clinvar, axis=1)
df['clinvar_same_motif'] = df.progress_apply(
    anno_clinvar.anno_same_motif_vars, tabixfile=tbx_clinvar, axis=1)

Parising SpliceAI results

In [ ]:
logger.info('Parsing SpliceAI results...')

#5-1. Annotate Exon/Intron position information
df['ExInt_INFO'] = df.progress_apply(splaiparser.calc_exint_info, 
                                     db=db, 
                                     db_intron=db_intron, 
                                     axis=1)

#5-2. Relative exon location
df['prc_exon_loc'] = df.parallel_apply(posparser.calc_prc_exon_loc, axis=1)


#5-3. Predict splicing effects
df['Pseudoexon'] = df.progress_apply(
    splaiparser.pseudoexon_activation,
    thresholds=thresholds_SpliceAI_parser, 
    db_intron=db_intron,
    axis=1)

df['Part_IntRet'] = df.parallel_apply(
    splaiparser.partial_intron_retention,
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)

df['Part_ExDel'] = df.parallel_apply(
    splaiparser.partial_exon_deletion,
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)

df['Exon_skipping'] = df.parallel_apply(
    splaiparser.exon_skipping, 
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)
                                        
df['Int_Retention'] = df.parallel_apply(
    splaiparser.intron_retention, 
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)

df['multiexs'] = df.parallel_apply(
    splaiparser.multi_exon_skipping, 
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)


Annotate aberrant splicing size

In [ ]:
logger.info('Annotating aberrant splicing size (bp)...')
#6-1. Annotate size of 
df['Size_Part_ExDel'] = df.parallel_apply(
    splaiparser.anno_partial_exon_del_size, 
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)

#6-2. Annotate size of partial intron retention
df['Size_Part_IntRet'] = df.parallel_apply(
    splaiparser.anno_partial_intron_retention_size, 
    thresholds=thresholds_SpliceAI_parser,
    axis=1)

#6-3. Annotate size of pseudoexon
df['Size_pseudoexon'] = df.parallel_apply(
    splaiparser.anno_gained_exon_size, 
    thresholds=thresholds_SpliceAI_parser, 
    axis=1)

#6-4. Annotate size of intron retention
df['Size_IntRet'] = df.parallel_apply(
    splaiparser.anno_intron_retention_size, 
    thresholds=thresholds_SpliceAI_parser,
    axis=1)

#6-5. Annotate size of exon skipping
df['Size_skipped_exon'] = df.parallel_apply(
    splaiparser.anno_skipped_exon_size, 
    thresholds=thresholds_SpliceAI_parser,
    axis=1)


Evaluate splicing effects

In [ ]:
logger.info('Predicting CDS change...')
#7-1. Predict CDS change
df['CDS_Length'] = df.progress_apply(predeffect.calc_cds_len, db=db, axis=1)
df['is_10%_truncation'] = df.progress_apply(predeffect.calc_cds_len_shorten, axis=1)

#7-2. Determine if the gene is included in eLoFs genes
df['is_eLoF'] = df.parallel_apply(predeffect.elofs_judge, axis=1)

#7-3. Determine causing NMD or not
df['is_NMD_at_Canon'] = df.parallel_apply(predeffect.nmd_judge, axis=1)

#7-4. Frame check
df['is_Frameshift_Part_ExDel'] = df['Size_Part_ExDel'].parallel_apply(
    predeffect.frame_check)
df['is_Frameshift_Part_IntRet'] = df['Size_Part_IntRet'].parallel_apply(
    predeffect.frame_check)
df['is_Frameshift_pseudoexon'] = df['Size_pseudoexon'].parallel_apply(
    predeffect.frame_check)
df['is_Frameshift_IntRet'] = df['Size_IntRet'].parallel_apply(
    predeffect.frame_check)
df['is_Frameshift_skipped_exon'] = df['Size_skipped_exon'].parallel_apply(
    predeffect.frame_check)
df['is_Frameshift'] = df[['is_Frameshift_Part_ExDel', 
                          'is_Frameshift_Part_IntRet', 
                          'is_Frameshift_pseudoexon', 
                          'is_Frameshift_IntRet', 
                          'is_Frameshift_skipped_exon'
                          ]].any(axis=1)


CCRs

In [ ]:
logger.info('Annotating CCRs info...')

#8-1. Annotate truncated regions 
df['skipped_region'] = df.parallel_apply(
    splaiparser.anno_skipped_regions, axis=1)

df['deleted_region'] = df.parallel_apply(
    splaiparser.anno_deleted_regions, 
    thresholds=thresholds_SpliceAI_parser, axis=1)

#8-2. Intersect with CCRs
logger.info('Annotate CCR scores...')
df = predeffect.anno_ccr_score(df)


Annotate priority scores

In [ ]:
logger.info('Annotating priority scores...')
scoring = Scoring(ths=sccore_ths)
df = df.astype({'maxsplai': 'float', 'vqslod': 'float', 
                'denovogear': float, 'triodenovo': float, 'dnmfilter': float})
df['insilico_screening'] = df.parallel_apply(scoring.insilico_screening, axis=1)
df['clinvar_screening'] = df.parallel_apply(scoring.clinvar_screening, axis=1)
df = scoring.calc_priority_score(df)

Display results

In [ ]:
df